In [26]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get_all("vbim-akqf", where="(hosp_yn != 'Unknown') and (hosp_yn != 'Missing') and (icu_yn != 'Unknown') and (icu_yn != 'Missing') and (death_yn != 'Unknown') and (death_yn != 'Missing') and (sex != 'Unknown') and (sex != 'Missing') and (sex != 'Other') and (sex != 'NA') and (age_group != 'Missing') and (age_group != 'NA') and (race_ethnicity_combined != 'Missing') and  (race_ethnicity_combined != 'NA') and (race_ethnicity_combined != 'Unknown') and (medcond_yn != 'Missing') and (medcond_yn != 'Unknown')", limit=22500000)
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [27]:
new_df = df.iloc[:,5:12]

print(new_df.head())
print(new_df.shape)

      sex    age_group race_ethnicity_combined hosp_yn icu_yn death_yn  \
0  Female  0 - 9 Years     White, Non-Hispanic      No     No       No   
1  Female  0 - 9 Years     White, Non-Hispanic      No     No       No   
2  Female  0 - 9 Years     White, Non-Hispanic      No     No       No   
3  Female  0 - 9 Years     White, Non-Hispanic      No     No       No   
4  Female  0 - 9 Years     White, Non-Hispanic      No     No       No   

  medcond_yn  
0        Yes  
1         No  
2         No  
3         No  
4        Yes  
(642408, 7)


In [28]:
print(new_df.sex.value_counts(), '\n')
print(new_df.age_group.value_counts(), '\n')
print(new_df.race_ethnicity_combined.value_counts(), '\n')
print(new_df.medcond_yn.value_counts(), '\n')

Female    342353
Male      300055
Name: sex, dtype: int64 

20 - 29 Years    103062
50 - 59 Years     97649
40 - 49 Years     91822
30 - 39 Years     91432
60 - 69 Years     82583
10 - 19 Years     61168
70 - 79 Years     53846
80+ Years         39577
0 - 9 Years       21269
Name: age_group, dtype: int64 

White, Non-Hispanic                                     399689
Hispanic/Latino                                         127430
Black, Non-Hispanic                                      66657
Multiple/Other, Non-Hispanic                             23417
Asian, Non-Hispanic                                      19514
Native Hawaiian/Other Pacific Islander, Non-Hispanic      3535
American Indian/Alaska Native, Non-Hispanic               2166
Name: race_ethnicity_combined, dtype: int64 

No     332902
Yes    309506
Name: medcond_yn, dtype: int64 



In [29]:
print(df.hosp_yn.value_counts(), '\n')
print(df.icu_yn.value_counts(), '\n')
print(df.death_yn.value_counts(), '\n')

No     523988
Yes    118420
Name: hosp_yn, dtype: int64 

No     603347
Yes     39061
Name: icu_yn, dtype: int64 

No     601211
Yes     41197
Name: death_yn, dtype: int64 



In [30]:
features = new_df.iloc[:,[0,1,2,6]]
labels = new_df.iloc[:,[3,4,5]]

print(features.head(), '\n')
print(labels.head(), '\n')

      sex    age_group race_ethnicity_combined medcond_yn
0  Female  0 - 9 Years     White, Non-Hispanic        Yes
1  Female  0 - 9 Years     White, Non-Hispanic         No
2  Female  0 - 9 Years     White, Non-Hispanic         No
3  Female  0 - 9 Years     White, Non-Hispanic         No
4  Female  0 - 9 Years     White, Non-Hispanic        Yes 

  hosp_yn icu_yn death_yn
0      No     No       No
1      No     No       No
2      No     No       No
3      No     No       No
4      No     No       No 



In [31]:
pd.options.mode.chained_assignment = None
features["sex"] = features["sex"].astype('category').cat.codes
features["age_group"] = features["age_group"].astype('category').cat.codes
features = pd.get_dummies(features, columns=["race_ethnicity_combined"], prefix=["race"])
features["medcond_yn"] = features["medcond_yn"].astype('category').cat.codes
print(features.head())


   sex  age_group  medcond_yn  \
0    0          0           1   
1    0          0           0   
2    0          0           0   
3    0          0           0   
4    0          0           1   

   race_American Indian/Alaska Native, Non-Hispanic  race_Asian, Non-Hispanic  \
0                                                 0                         0   
1                                                 0                         0   
2                                                 0                         0   
3                                                 0                         0   
4                                                 0                         0   

   race_Black, Non-Hispanic  race_Hispanic/Latino  \
0                         0                     0   
1                         0                     0   
2                         0                     0   
3                         0                     0   
4                         0                     0

In [32]:
labels["hosp_yn"] = labels["hosp_yn"].astype('category').cat.codes
labels["icu_yn"] = labels["icu_yn"].astype('category').cat.codes
labels["death_yn"] = labels["death_yn"].astype('category').cat.codes
print(labels.head())

   hosp_yn  icu_yn  death_yn
0        0       0         0
1        0       0         0
2        0       0         0
3        0       0         0
4        0       0         0


In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features.to_numpy(), labels.to_numpy(), test_size=0.2, random_state=42)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import normalize
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)

In [35]:
modelHosp = Sequential()
modelHosp.add(Dense(10, input_dim=features.shape[1], activation='relu'))
modelHosp.add(Dense(10, activation='relu'))
modelHosp.add(Dense(1, activation='sigmoid'))
modelHosp.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
modelHosp.fit(x_train,y_train,validation_data=(x_test,y_test[:,0]),epochs=20)

Epoch 1/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0848 - accuracy: 0.8961 - val_loss: 0.1225 - val_accuracy: 0.8199
Epoch 2/20
16061/16061 [==============================] - 16s 1ms/step - loss: 0.0767 - accuracy: 0.8981 - val_loss: 0.1254 - val_accuracy: 0.8216
Epoch 3/20
16061/16061 [==============================] - 16s 1ms/step - loss: 0.0764 - accuracy: 0.8984 - val_loss: 0.1274 - val_accuracy: 0.8197
Epoch 4/20
16061/16061 [==============================] - 16s 1ms/step - loss: 0.0764 - accuracy: 0.8981 - val_loss: 0.1205 - val_accuracy: 0.8236
Epoch 5/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0763 - accuracy: 0.8985 - val_loss: 0.1273 - val_accuracy: 0.8216
Epoch 6/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0765 - accuracy: 0.8981 - val_loss: 0.1284 - val_accuracy: 0.8181
Epoch 7/20
16061/16061 [==============================] - 16s 992us/step - loss: 0.0763 - accuracy: 0.8983 - val_loss:

In [36]:
modelIcu = Sequential()
modelIcu.add(Dense(10, input_dim=features.shape[1], activation='relu'))
modelIcu.add(Dense(10, activation='relu'))
modelIcu.add(Dense(1, activation='sigmoid'))
modelIcu.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
modelIcu.fit(x_train,y_train,validation_data=(x_test,y_test[:,1]),epochs=20)

Epoch 1/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0822 - accuracy: 0.8976 - val_loss: 0.0609 - val_accuracy: 0.9313
Epoch 2/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0770 - accuracy: 0.8979 - val_loss: 0.0550 - val_accuracy: 0.9362
Epoch 3/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0765 - accuracy: 0.8983 - val_loss: 0.0554 - val_accuracy: 0.9361
Epoch 4/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0765 - accuracy: 0.8983 - val_loss: 0.0552 - val_accuracy: 0.9359
Epoch 5/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0763 - accuracy: 0.8986 - val_loss: 0.0580 - val_accuracy: 0.9339
Epoch 6/20
16061/16061 [==============================] - 16s 988us/step - loss: 0.0765 - accuracy: 0.8981 - val_loss: 0.0547 - val_accuracy: 0.9349
Epoch 7/20
16061/16061 [==============================] - 16s 984us/step - loss: 0.0763 - accuracy: 0.8984 - val_los

In [37]:
modelDeath = Sequential()
modelDeath.add(Dense(10, input_dim=features.shape[1], activation='relu'))
modelDeath.add(Dense(10, activation='relu'))
modelDeath.add(Dense(1, activation='sigmoid'))
modelDeath.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
modelDeath.fit(x_train,y_train,validation_data=(x_test,y_test[:,2]),epochs=20)

Epoch 1/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0894 - accuracy: 0.8857 - val_loss: 0.0515 - val_accuracy: 0.9352
Epoch 2/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0767 - accuracy: 0.8982 - val_loss: 0.0498 - val_accuracy: 0.9351
Epoch 3/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0766 - accuracy: 0.8979 - val_loss: 0.0498 - val_accuracy: 0.9352
Epoch 4/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0763 - accuracy: 0.8987 - val_loss: 0.0513 - val_accuracy: 0.9352
Epoch 5/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0765 - accuracy: 0.8982 - val_loss: 0.0490 - val_accuracy: 0.9359
Epoch 6/20
16061/16061 [==============================] - 17s 1ms/step - loss: 0.0764 - accuracy: 0.8984 - val_loss: 0.0502 - val_accuracy: 0.9351
Epoch 7/20
16061/16061 [==============================] - 18s 1ms/step - loss: 0.0763 - accuracy: 0.8982 - val_loss: 0

In [38]:
test_loss, test_acc = model.evaluate(x=x_test, y=y_test) 
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

4016/4016 [==============================] - 3s 741us/step - loss: 0.0705 - accuracy: 0.9912
Test Loss: 0.07053086161613464, Test Accuracy: 0.9912205338478088


In [39]:
import os
modelHosp.save(os.path.join("./model/","covid_model_hosp.h5"))
modelIcu.save(os.path.join("./model/","covid_model_icu.h5"))
modelDeath.save(os.path.join("./model/","covid_model_death.h5"))